In [1]:
import pandas as pd
import numpy as np

# Gerar Inserts Projeto BD1

## pré-processar dados

### Carregar dados no dataframe disciplina e pré-processar colunas

In [2]:
disciplina = pd.read_excel("dados/disciplinasDCET.xlsx")

disciplina['COD'] = disciplina['COD'].replace(" ", "")
disciplina.columns = disciplina.columns.str.lower()
disciplina = disciplina.rename(columns={"b ou l": "formacao", "área": "area"})
disciplina['id_departamento'] = 5
disciplina.loc[disciplina['area'].isna(), 'area'] = 'Oceanografia'

# Replace some Nan values
disciplina.loc[407, 'ch'] = 60
disciplina.loc[559, 'ch'] = 90
disciplina.loc[730, 'ch'] = 60

columns = ['area', 'curso', 'disciplina', 'formacao']

for column in columns:
    disciplina[column] = disciplina[column].str.lower().str.strip()

mask = disciplina['formacao'] == 'b e l'
disciplina.loc[mask, 'formacao'] = 'b+l'

disciplina.loc[disciplina['area'] == 'dcet', 'area'] = 'Oceanografia'

mask = disciplina['formacao'] == 'b+l'
tmp = disciplina.loc[mask].copy()
tmp.formacao = 'l'
disciplina.loc[mask, 'formacao'] = 'b'
disciplina = pd.concat([disciplina, tmp], axis=0).reset_index(drop=True)

disciplina

,cod,disciplina,ch,formacao,curso,area,id_departamento
0,CET154,química geral e orgânica,90.0,b,biomedicina,química,5
1,CET558,calculo aplicado a biomedicina,60.0,b,biomedicina,matemática,5
2,CET559,física aplicada a biomedicina,60.0,b,biomedicina,física,5
3,CIB520,bioestatistica,60.0,b,biomedicina,estatística,5
4,CIB517,bioinformática,60.0,b,biomedicina,computação,5
...,...,...,...,...,...,...,...
802,CET020,bioestatística,60.0,l,ciências biológicas,estatística,5
803,CET561,matemática para biociências,60.0,l,ciências biológicas,matemática,5
804,CET647,fisíca para biociências,45.0,l,ciências biológicas,física,5
805,CET650,quimíca para biociências,60.0,l,ciências biológicas,química,5


### Carregar dataframes departamentos, docentes, pro_reitoria, semestre e colegiados

In [3]:
departamento = pd.read_csv("dados/departamentos.csv")
departamento.sigla = departamento.sigla.str.lower()
departamento.nome = departamento.nome.str.lower()
departamento.head()

,id,sigla,nome
0,1,dcaa,departamento de ciências agrárias e ambientais
1,2,dcac,departamento de administração e ciências contá...
2,3,dcb,departamento de ciências biológicas
3,4,dcec,departamento de ciências econômicas
4,5,dcet,departamento de ciências exatas e tecnológicas


In [4]:
docente = pd.read_csv("dados/docentes.csv")
docente.nome = docente.nome.str.lower()
docente.head()

,id,nome,cargo,titulo,id_departamento,id_area
0,1,álvaro vinícius de souza coelho,titular,doutor,5,1
1,2,antônio henrique figueira louro,adjunto,doutor,5,1
2,3,aprígio augusto lopes bezerra,adjunto,doutor,5,1
3,4,césar alberto bravo pariente,adjunto,doutor,5,1
4,5,dany sanchez dominguez,pleno,doutor,5,1


In [5]:
pro_reitoria = pd.read_csv("dados/pro_reitorias.csv")
pro_reitoria.sigla = pro_reitoria.sigla.str.lower()
pro_reitoria.nome = pro_reitoria.nome.str.lower()
pro_reitoria

,id,sigla,nome
0,1,propp,pró-reitoria de pesquisa e pós-graduação
1,2,prograd,pró-reitoria de graduação
2,3,proex,pró-reitoria de extensão
3,4,proad,pró-reitoria administrativa
4,5,projur,procuradoria jurídica


In [6]:
semestre = pd.read_csv("dados/semestres.csv")
semestre.semestre = semestre.semestre.astype("str")
semestre

,id,semestre,id_pro_reitoria
0,1,2019.1,2
1,2,2019.2,2
2,3,2020.1,2
3,4,2020.2,2
4,5,2021.1,2
5,6,2021.2,2
6,7,2022.1,2
7,8,2022.2,2


In [7]:
colegiado = pd.read_csv("dados/colegiados.csv")
colegiado.colegiado = colegiado.colegiado.str.lower()
colegiado.head()

,id,colegiado
0,1,administração
1,2,agronomia
2,3,biomedicina
3,4,ciências biológicas
4,5,ciências contábeis


### Criar Dataframes curso, tipo_formacao e area

In [8]:
curso = pd.DataFrame(disciplina['curso']).drop_duplicates().reset_index().rename(columns={"index": "id"})
curso = curso.sort_values(by=["curso"]).reset_index(drop=True) 
curso['id'] = np.arange(1, curso.shape[0] + 1)
id_colegiado = [1, 2, 3, 4, 5, 7, 9, 6, 12, 13, 15, 16, 14, 17, 19, 20, 22, 24, 26, 28]
curso['id_colegiado'] = id_colegiado
curso.head()

,id,curso,id_colegiado
0,1,administração,1
1,2,agronomia,2
2,3,biomedicina,3
3,4,ciências biológicas,4
4,5,ciências contábeis,5


In [9]:
formacao = pd.DataFrame(disciplina['formacao']).drop_duplicates()
formacao = formacao.reset_index().rename(columns={"index": "id"})
formacao['id'] = np.arange(1, formacao.shape[0] + 1)
formacao

,id,formacao
0,1,b
1,2,l


In [10]:
area = pd.DataFrame(disciplina['area']).drop_duplicates().reset_index().rename(columns={"index": "id"})
area = area.sort_values(by="area").reset_index(drop=True)
area['id'] = np.arange(1, area.shape[0] + 1)
area.head()

,id,area
0,1,Oceanografia
1,2,computação
2,3,eng. civil
3,4,eng. elétrica
4,5,eng. mecânica


### Criar colunas PK e FK

In [11]:
def create_fk(df_fk, column_fk):
    fk_values = {df_fk.loc[i, column_fk]: i + 1 for i in range(df_fk.shape[0])}
    return fk_values
        
disciplina.curso = disciplina.curso.map(create_fk(curso, 'curso'))
disciplina.area = disciplina.area.map(create_fk(area, 'area'))
disciplina.formacao = disciplina.formacao.map(create_fk(formacao, 'formacao'))

tmp = pd.DataFrame({"cod":disciplina.cod.unique()})
index = pd.DataFrame({"id": disciplina.cod.map(create_fk(tmp, 'cod'))})
disciplina = pd.concat([index, disciplina], axis=1)

new_names_columns = {"area": "id_area", "curso": "id_curso", "formacao": "id_formacao"}
disciplina = disciplina.rename(columns=new_names_columns)

### Criar DataFrame disciplina_formacao, curso_formacao, sem_disc, sem_disc_docente e sem_disc_curso

In [12]:
disciplina_formacao = disciplina[['id', 'id_formacao']].reset_index(drop=True)
disciplina_formacao = disciplina_formacao.rename(columns={"id": "id_disciplina"})
disciplina_formacao.head()

,id_disciplina,id_formacao
0,1,1
1,2,1
2,3,1
3,4,1
4,5,1


In [13]:
curso_formacao = disciplina[['id_curso', 'id_formacao']].reset_index(drop=True)
curso_formacao.head()

,id_curso,id_formacao
0,3,1
1,3,1
2,3,1
3,3,1
4,3,1


In [14]:
id_disciplina = []
id_docente = []

with open("dados/disciplina_docente.txt") as file:
    for line in file:
        line = line.split(" ")
        id_disciplina.append(int(line[0]))
        id_docente.append(int(line[1].replace("\n", "")))

sem_disc = disciplina.loc[disciplina.id.isin(id_disciplina), ::].copy(deep=True) # disciplinas da área computação
sem_disc = sem_disc[sem_disc.id_curso == 7] # disciplinas apenas do curso de computação
sem_disc['id_semestre'] = 8 # inserir id do semestre

sem_disc = sem_disc[['id_semestre', 'id_curso', 'id']].reset_index(drop=True)

aux = pd.DataFrame({'id_disciplina': id_disciplina, 'id_docente': id_docente})
sem_disc = sem_disc.merge(aux, how='inner', left_on='id', right_on='id_disciplina').drop(columns='id')

sem_disc['id'] = np.arange(1, sem_disc.shape[0] + 1)

sem_disc_docente = sem_disc[['id', 'id_docente']].reset_index(drop=True).rename(columns={'id':'id_sem_disc'})
sem_disc_curso = sem_disc[['id', 'id_curso']].reset_index(drop=True).rename(columns={'id':'id_sem_disc'})
sem_disc = sem_disc[['id', 'id_semestre', 'id_disciplina']].reset_index(drop=True)
sem_disc

,id,id_semestre,id_disciplina
0,1,8,526
1,2,8,535
2,3,8,537
3,4,8,539
4,5,8,546
5,6,8,548
6,7,8,549
7,8,8,550
8,9,8,428
9,10,8,555


In [15]:
sem_disc_curso

,id_sem_disc,id_curso
0,1,7
1,2,7
2,3,7
3,4,7
4,5,7
5,6,7
6,7,7
7,8,7
8,9,7
9,10,7


In [16]:
sem_disc_docente

,id_sem_disc,id_docente
0,1,22
1,2,4
2,3,4
3,4,4
4,5,7
5,6,4
6,7,1
7,8,6
8,9,13
9,10,5


### Finalizar pré-processamento disciplina

In [17]:
disciplina = disciplina.drop(["id_formacao","id_curso"], axis=1).drop_duplicates(subset="cod").reset_index(drop=True)
id_area = disciplina.id_area.copy(deep=True)
disciplina = disciplina.drop(columns='id_area')
disciplina = pd.concat([disciplina, id_area], axis=1)
disciplina.head()

,id,cod,disciplina,ch,id_departamento,id_area
0,1,CET154,química geral e orgânica,90.0,5,11
1,2,CET558,calculo aplicado a biomedicina,60.0,5,10
2,3,CET559,física aplicada a biomedicina,60.0,5,9
3,4,CIB520,bioestatistica,60.0,5,8
4,5,CIB517,bioinformática,60.0,5,2


## Gerar e Salvar inserts

In [18]:
def create_insert(df, nome_tabela) -> list:
    inserts = []
    
    for i in range(df.shape[0]):
        values = []
        atributos = []
        for column in df.columns:
            atributos.append(f"{column}")
                
            if isinstance(df.loc[i, column], str):
                values.append(f"'{df.loc[i, column]}'")
            else:
                values.append(f"{df.loc[i, column]}")
                
        values = ", ".join(values)
        atributos = ", ".join(atributos)
        insert = f"insert into {nome_tabela} ({atributos}) values ({values});\n"
        inserts.append(insert)
    return inserts

inserts = create_insert(departamento, "departamento")
inserts += create_insert(pro_reitoria, "pro_reitoria")
inserts += create_insert(semestre, "semestre")
inserts += create_insert(formacao, "formacao")
inserts += create_insert(colegiado, "colegiado")
inserts += create_insert(curso, "curso")
inserts += create_insert(area, "area")
inserts += create_insert(disciplina, "disciplina")
inserts += create_insert(docente, "docente")
inserts += create_insert(disciplina_formacao, "disciplina_formacao")
inserts += create_insert(curso_formacao, "curso_formacao")
inserts += create_insert(sem_disc, "sem_disc")
inserts += create_insert(sem_disc_curso, "sem_disc_curso")
inserts += create_insert(sem_disc_docente, "sem_disc_docente")

with open("sql/inserts.sql", "w") as file:
    for insert in inserts:
        file.write(insert)